# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [33]:
from datetime import datetime

from pyspark.sql import SparkSession

from pyspark.sql.functions import min as smin, max as smax, sum as ssum, round as sround
from pyspark.sql.functions import isnan, isnull, when, first, avg, last, count, countDistinct, col, lag, lead, coalesce, lit, split, trim

from pyspark.sql.window import Window
from pyspark.sql.functions import to_date, date_format, from_unixtime, to_timestamp

from pyspark.sql.types import DateType, TimestampType, IntegerType
 
import jupyter_utils as j

from pyspark import SparkContext

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StandardScaler
 
SparkContext.setSystemProperty('spark.logConf', 'True')
SparkContext.setSystemProperty('spark.default.parallelism', '16')
SparkContext.setSystemProperty('spark.executor.memory', '4g')
SparkContext.setSystemProperty('spark.driver.memory', '8g')
SparkContext.setSystemProperty('spark.reducer.maxSizeInFlight', '96m')
SparkContext.setSystemProperty('spark.shuffle.consolidateFiles', 'True') 
SparkContext.setSystemProperty('spark.shuffle.service.index.cache.size', '500m')

SparkContext.setSystemProperty('spark.driver.extraJavaOptions', '-server -Xmx8G')
SparkContext.setSystemProperty('spark.executor.extraJavaOptions', '-server -Xmx8G -XX:+UseG1GC')

In [2]:
j.reload(j)

In [3]:
# filepath = 'sparkify_full_csv_data.csv'
filepath = 'medium_sparkify_event_data.json'

In [4]:
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

spark.sparkContext.setLogLevel('INFO')

In [5]:
spark.sparkContext.getConf().getAll()

[('spark.executor.memory', '4g'),
 ('spark.executor.id', 'driver'),
 ('spark.app.id', 'local-1584893128494'),
 ('spark.driver.port', '43275'),
 ('spark.shuffle.service.index.cache.size', '500m'),
 ('spark.reducer.maxSizeInFlight', '96m'),
 ('spark.shuffle.consolidateFiles', 'True'),
 ('spark.default.parallelism', '16'),
 ('spark.logConf', 'True'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.memory', '8g'),
 ('spark.app.name', 'Sparkify'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', '192.168.0.107')]

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [6]:
# df = spark.read.option("inferSchema", "true").option("header", "true").option("encoding", "utf-8").csv(filepath)
df = spark.read.option("inferSchema", "true").option("header", "true").option("encoding", "utf-8").json(filepath)

In [33]:
from pyspark import StorageLevel

print(df.storageLevel)

df.persist(StorageLevel.MEMORY_ONLY)

print(df.storageLevel)

Serialized 1x Replicated
Memory Serialized 1x Replicated


In [ ]:
df.printSchema()

In [ ]:
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

In [ ]:
df.groupby('auth').agg(count(col('auth'))).show(50)

In [ ]:
df.where(~df.auth.isin(['Logged In', 'Cancelled'])).show(1)

In [ ]:
df.where((df.auth == 'Logged In') & (df.page == 'Home')).show(10)

In [7]:
log4jLogger = spark.sparkContext._jvm.org.apache.log4j

LOGGER = log4jLogger.LogManager.getLogger('driver_logger')

def info(message, print_on_notebook = True):
    LOGGER.info(message)
    
    if print_on_notebook:
        print(message)
    
info('Logger instance created')

Logger instance created


In [8]:
CHURN_CANCELLATION_PAGE = 'Cancellation Confirmation'
REGISTRATION_PAGE = 'Submit Registration'
milliseconds_to_hours = 3600 * 1000
minutes_to_hours = 60 * 60
TRUE = 1
FALSE = 0

def clean_dataframe(df):
    
    info('Starting data cleaning...')
    
    total_before = df.count()
    
    # Keep only logged records
    df = df.where(df.auth.isin(['Logged In', 'Cancelled']))
    
    # Records without userId
    df = df.where(col('userId').isNotNull())
    
    # Create a date column for the event
    df = df.withColumn('date', from_unixtime(col('ts') / 1000).cast(DateType()))
    
    # Location
    # df = df.withColumn('state', trim(split((split('location', ',').getItem(1)), '-').getItem(0)))
    
    # Relevant windows
    w_session = Window.partitionBy('sessionId').orderBy('ts')
    w_user_session = Window.partitionBy('sessionId', 'userId').orderBy('ts').rangeBetween(Window.unboundedPreceding, Window.unboundedFollowing)
    w_user = Window.partitionBy('userId').orderBy('ts').rangeBetween(Window.unboundedPreceding, Window.unboundedFollowing)
    
    # Create features
    df = df.withColumn('previous_page', lag(df.page).over(w_session))
    df = df.withColumn('last_event_ts', last(col('ts')).over(w_user))
    df = df.withColumn('last_page', last(col('page')).over(w_user))
    df = df.withColumn('register_page', first(col('previous_page')).over(w_user))
    df = df.withColumn('first_ts', first(col('ts')).over(w_user))
    df = df.withColumn('ts_elapsed', last(df.ts).over(w_session) - first(df.ts).over(w_user_session))
    df = df.withColumn('session_duration', smax(df.ts_elapsed).over(w_user_session))
     
    info('Finished data cleaning...')
    info(f'Number of removed rows: {total_before - df.count()}')
    
    return df

In [9]:
df = clean_dataframe(df)

Starting data cleaning...
Finished data cleaning...
Number of removed rows: 15700


# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [ ]:
df.groupBy('page').count().orderBy('count', ascending = False).show(50)

Some questions about the data:

- Are errors related to downgrading canceling the service?
- Having a certain number of friends or a sense of community can decrease the churn?
- Thumbs down are related to churn? (could the quality of the songs catalog affect the churn)
- The advertising is not annoying the users?
- Users with stay connected for more time have less change to churn?
- Is the home page relevant?
- Users, who access the downgrade page are how much more willing to churn?

In [ ]:
df.groupBy('status').count().orderBy('count', ascending = False).show(20)

In [ ]:
df.filter('userId = 92').groupBy('page').count().orderBy('count', ascending = False).show(50)

In [ ]:
df.filter('userId = 92').groupBy('page').count().orderBy('count', ascending = False).show(50)

In [ ]:
df.filter('userId = 92').groupBy('userAgent').count().orderBy('count', ascending = False).show(50, False)

In [ ]:
df.filter('userId = 92 and song != \'null\' ').groupBy('song').count().orderBy('count', ascending = False).show(50, False)

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [10]:
def create_session_dimension(df):
    
    # sessions from the user
    df_sessions = df.orderBy(df.sessionId).groupBy('sessionId', 'userId').agg(
        smax(df.ts).alias('max_event_ts'),
        smin(df.ts).alias('min_event_ts'),
        ssum(df.length).alias('session_n_total_playback'), # Based on songs length
        count(when(df.page == 'Thumbs Up', True)).alias("session_n_likes"),
        count(when(df.page == 'Thumbs Down', True)).alias("session_n_dislikes"),
        count(when(df.page == 'NextSong', True)).alias("session_n_songs"),
        count(when(df.page == 'Add Friend', True)).alias("session_n_friends"),
        count(when(df.page == 'Add to Playlist', True)).alias("session_n_add_playlist"),
        count(when(df.page == 'Home', True)).alias("session_n_home"),
        count(when(df.page == 'Roll Advert', True)).alias("session_n_ads"),
        count(when(df.page == 'Help', True)).alias("session_n_help"),
        count(when(df.page == 'Error', True)).alias("session_n_error"),
        count(when(df.page == 'Settings', True)).alias("session_n_sets"),
        count(col('page')).alias('session_n_actions'),
        first(col('session_duration')).alias('session_duration')
    ) 
    
    # Calculate the interval until the next session
    w_user_sessions_interval = Window.partitionBy('userId').orderBy('min_event_ts')
    df_sessions = df_sessions.withColumn('interval_to_session', col('min_event_ts') - lag(col('max_event_ts')).over(w_user_sessions_interval))
    
    # Calculate average time in hours for each session
    df_session_time = df_sessions.groupBy('userId').agg(
       (avg(df_sessions.session_duration) / milliseconds_to_hours).alias('session_hours')
    )
    df_sessions = df_sessions.join(df_session_time, on = 'userId')
    
    # We should remove the null lines before count/group to not account 2 times the mean interval
    df_sessions = df_sessions.groupBy('userId').agg(  
        (avg(df_sessions.interval_to_session) / milliseconds_to_hours).alias('session_avg_time_away'),
        ((avg(df_sessions.session_n_total_playback) / minutes_to_hours) / first(col('session_hours'))).alias('session_avg_playback'), 
        (avg(df_sessions.session_n_likes) / first(col('session_hours'))).alias('session_avg_likes'),
        (avg(df_sessions.session_n_dislikes) / first(col('session_hours'))).alias('session_avg_dislikes'),
        (avg(df_sessions.session_n_songs) / first(col('session_hours'))).alias('session_avg_songs'),
        (avg(df_sessions.session_n_friends) / first(col('session_hours'))).alias('session_avg_friends'),
        (avg(df_sessions.session_n_add_playlist) / first(col('session_hours'))).alias('session_avg_added_playlist'),
        (avg(df_sessions.session_n_home) / first(col('session_hours'))).alias('session_avg_home'),
        (avg(df_sessions.session_n_ads) / first(col('session_hours'))).alias('session_avg_ads'),
        (avg(df_sessions.session_n_help) / first(col('session_hours'))).alias('session_avg_help'),
        (avg(df_sessions.session_n_error) / first(col('session_hours'))).alias('session_avg_errors'),
        (avg(df_sessions.session_n_sets) / first(col('session_hours'))).alias('session_avg_settings'),
        (avg(df_sessions.session_n_actions) / first(col('session_hours'))).alias('session_avg_actions')
    )
    
    return df_sessions

def create_user_dimension(df):
    
    df_user_profile = df.groupby('userId')\
        .agg( 

            # first(col('state')).alias('state'),
            first(when(col('gender') == 'M', TRUE).otherwise(FALSE)).alias('male'),

            smin(col('first_ts')).alias('ts_start'),
            smax(col('last_event_ts')).alias('ts_end'),        
        
            ((smax(col('last_event_ts')) - smin(col('first_ts'))) / milliseconds_to_hours).alias('time_window'),
        
            # Subscription
            count(when(col('page') == 'Submit Downgrade', True)).alias('n_downgrades'),
            count(when(col('page') == 'Submit Upgrade', True)).alias('n_upgrades'),
            last(when(col('level') == 'paid', TRUE).otherwise(FALSE)).alias('paid'),
            first(when(col('last_page') == CHURN_CANCELLATION_PAGE, TRUE).otherwise(FALSE)).alias('canceled'),

            # Streaming
            count(when(col('page') == 'NextSong', True)).alias('n_songs'),
            count(when(col('page') == 'Thumbs Up', True)).alias('n_likes'),
            count(when(col('page') == 'Thumbs Down', True)).alias('n_dislikes'),
            countDistinct(col('sessionId')).alias('n_sess'),
            (avg(col('session_duration')) / milliseconds_to_hours).alias('avg_session_duration'),

            # Community
            count(when(col('page') == 'Add Friend', True)).alias('n_friends'),
            count(when(col('page') == 'Add to Playlist', True)).alias('n_added_to_playlist'),

            # Other
            count(when(col('page') == 'Home', True)).alias('n_home'),
            count(when(col('page') == 'Roll Advert', True)).alias('n_ads'),
            count(when(col('page') == 'Help', True)).alias('n_help'),
            count(when(col('page') == 'Error', True)).alias('n_errors'),
            count(when(col('page') == 'Settings', True)).alias('n_settings'),
            count(col('page')).alias('n_actions')
        )
    
    
    # Location
    # states = list(map(lambda c: c[0].strip(), df.select(['state']).distinct().rdd.collect()))
    # for state in states:
    #    df_user_profile = df_user_profile.withColumn(state.lower(), when(df_user_profile.state == state, 1).otherwise(0))
    
    return df_user_profile

def create_days_dimension(df):
    
    df_unique_days = df.groupby('userId').agg(countDistinct('date').alias('n_days'))
    
    df_daily_actions = df.groupby('userId', 'date').agg(count('page').alias('total'))
    df_daily_actions = df_daily_actions.groupby('userId').agg(avg('total').alias('avg_daily_actions')) 

    df_days = df_unique_days.join(df_daily_actions, df_unique_days.userId == df_daily_actions.userId)
    
    # Remove duplicated column after join
    df_days = df_days.drop(df_daily_actions.userId)
    
    return df_days

def sort_features(df, columns_order):
    _columns = df.columns
    _columns.sort()
    
    for _idx, _val in list(enumerate(columns_order)):
        _columns.pop(_columns.index(_val))
        _columns.insert(_idx, _val)
        
    assert len(_columns) == len(df.columns)

    return _columns

In [11]:
df_sessions = create_session_dimension(df)
df_days = create_days_dimension(df)

df_users = create_user_dimension(df)
df_users = df_users.orderBy(df_users.userId).join(df_days, on = 'userId')

_columns = sort_features(df_users, [ 'userId', 'male', 'paid', 'canceled'])
_columns = list(set(df_users.schema.names + df_sessions.schema.names) - set(['ts_start', 'ts_end', 'state']))

df_users = df_users.orderBy(df_users.userId).join(df_sessions, on = 'userId').select(_columns) 

In [ ]:
df_users.columns

In [ ]:
### WARN: Only round to display
# Enforces the order for some columns
df_users.select([sround(c, 0).cast(dataType = IntegerType()).alias(c) for c in _columns]).fillna(0).show(2, True, vertical = True)

In [ ]:
df_users.select(_columns).fillna(0).toPandas().to_csv('sparkify_data_final.csv', index = False)

In [ ]:
df.agg(countDistinct(df.userId).alias('unique_users')).show()

In [ ]:
df_users.orderBy(df_users.userId).join(df_sessions, on = 'userId').select(_columns).count()

In [ ]:
df_users.orderBy(df_users.userId).join(df_sessions, on = 'userId').select(_columns).groupBy('canceled').agg(count(df_users.canceled).alias('total')).show()

- Advertises number (per session and all)
    - The user **100010** returned after some idle time and received a considerable amount of advertises;
    - Also, after thumbs down, I received two advertisements on four sounds. Then canceled the service.
- Number of sessions
- Paid subscription time
- Avg songs before an ad
- Number of skipped songs

In [ ]:
df.schema.names

In [ ]:
to_date(df.ts.cast(dataType=TimestampType()))

In [ ]:
df.where(df.userId == user_id).select(['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level', 
 'page',
 'sessionId',
 'song', 
 'ts', 
 'userId']).orderBy('sessionId', 'itemInSession').withColumn('datetime', date_format((df.ts/1000).cast(dataType=TimestampType()), 'HH:mm:ss dd-MM-YYYY')).show(350, True)

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [12]:
from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.feature import StringIndexer, VectorAssembler

In [ ]:
columns_to_exclude = set(['userId'])

columns_to_use = list(set(df_users.columns) - columns_to_exclude)

columns_to_train = list(set(columns_to_use) - set(['canc']))

columns_to_use.sort()
columns_to_train.sort()

print(f'Columns: {columns_to_use}\n')
print(f'Columns to train: {columns_to_train}')

In [13]:
CHURN_LABEL = 'canceled'
TRAIN_SPLIT_RATIO = .7
TEST_SPLIT_RATIO = .3

SPLIT_RATIO = [TRAIN_SPLIT_RATIO, TEST_SPLIT_RATIO]

In [14]:
binary_features = [ 'paid', 'male' ]

numeric_features = [
    'avg_daily_actions',
    'avg_session_duration', 
    'n_actions',
    'n_added_to_playlist',
    'n_ads',
    'n_days',
    'n_dislikes',
    'n_downgrades',
    'n_errors',
    'n_friends',
    'n_help',
    'n_home',
    'n_likes',
    'n_sess',
    'n_settings',
    'n_songs',
    'n_upgrades', 
    'session_avg_actions',
    'session_avg_added_playlist',
    'session_avg_ads',
    'session_avg_dislikes',
    'session_avg_errors',
    'session_avg_friends',
    'session_avg_help',
    'session_avg_home',
    'session_avg_likes',
    'session_avg_playback',
    'session_avg_settings',
    'session_avg_songs',
    'session_avg_time_away',
    'time_window'
]

columns_all = [
    'canceled',
    'male',
    'paid',
    'avg_daily_actions',
    'avg_session_duration', 
    'n_actions',
    'n_added_to_playlist',
    'n_ads',
    'n_days',
    'n_dislikes',
    'n_downgrades',
    'n_errors',
    'n_friends',
    'n_help',
    'n_home',
    'n_likes',
    'n_sess',
    'n_settings',
    'n_songs',
    'n_upgrades', 
    'session_avg_actions',
    'session_avg_added_playlist',
    'session_avg_ads',
    'session_avg_dislikes',
    'session_avg_errors',
    'session_avg_friends',
    'session_avg_help',
    'session_avg_home',
    'session_avg_likes',
    'session_avg_playback',
    'session_avg_settings',
    'session_avg_songs',
    'session_avg_time_away',
    'time_window'
]

columns_to_train = [
    'male',
    'paid',
    'avg_daily_actions',
    'avg_session_duration', 
    'n_actions',
    'n_added_to_playlist',
    'n_ads',
    'n_days',
    'n_dislikes',
    'n_downgrades',
    'n_errors',
    'n_friends',
    'n_help',
    'n_home',
    'n_likes',
    'n_sess',
    'n_settings',
    'n_songs',
    'n_upgrades', 
    'session_avg_actions',
    'session_avg_added_playlist',
    'session_avg_ads',
    'session_avg_dislikes',
    'session_avg_errors',
    'session_avg_friends',
    'session_avg_help',
    'session_avg_home',
    'session_avg_likes',
    'session_avg_playback',
    'session_avg_settings',
    'session_avg_songs',
    'session_avg_time_away',
    'time_window'
]

In [15]:
def evaluate_multiclass_classifier(predictions, columns):
    metrics_to_evaluate = [ 'accuracy', 'f1', 'weightedPrecision', 'weightedRecall' ]
    
    result = {}
    for metric in metrics_to_evaluate:
        evaluator = MulticlassClassificationEvaluator(labelCol = columns[0], predictionCol = columns[1], metricName = metric)
        value = evaluator.evaluate(predictions)
        result[metric] = value
        print(f'{metric}: {value}') 
    
    return result

def train_random_forest_classifier(data, columns, train_cloumns):
    
    # Split train/test
    (train_df, test_df) = data.randomSplit(SPLIT_RATIO, seed = 42)
    
    # Create the indexer for labels
    l_indexer = StringIndexer(inputCol = CHURN_LABEL, outputCol = 'idx_labels')
    f_binaries = VectorAssembler(inputCols = binary_features, outputCol = 'bin_features')
    f_numeric = VectorAssembler(inputCols = numeric_features, outputCol = 'num_features')
    
    f_scaler = StandardScaler(inputCol = 'num_features', outputCol = 'num_features_escaled', withStd = True, withMean = True)
    
    f_all = VectorAssembler(inputCols = [ 'bin_features' , 'num_features_escaled' ], outputCol = 'features')
    
    l_translator = IndexToString(inputCol = 'prediction', outputCol = 'predictedLabel', labels = [ 'Not churn', 'Churn' ])
    
    rf_classifier = RandomForestClassifier(labelCol = 'idx_labels', featuresCol = 'features', numTrees = 10, maxBins = 5, impurity = 'entropy', minInstancesPerNode = 3, seed = 42)
    
    pipeline = Pipeline(stages = [ l_indexer, f_binaries, f_numeric, f_scaler, f_all, rf_classifier, l_translator ])
    
    # Train the model
    model = pipeline.fit(train_df)

    # Test the model
    predictions = model.transform(test_df)

    return model.stages[2], predictions
    

def create_pipeline(model):
    
    l_indexer = StringIndexer(inputCol = CHURN_LABEL, outputCol = 'idx_labels')
    f_binaries = VectorAssembler(inputCols = binary_features, outputCol = 'bin_features')
    f_numeric = VectorAssembler(inputCols = numeric_features, outputCol = 'num_features')
    f_scaler = StandardScaler(inputCol = 'num_features', outputCol = 'num_features_escaled', withStd = True, withMean = True)
    f_all = VectorAssembler(inputCols = [ 'bin_features' , 'num_features_escaled' ], outputCol = 'features')
    pipeline = Pipeline(stages = [ l_indexer, f_binaries, f_numeric, f_scaler, f_all, model ])
    
    return pipeline

def create_random_forest_pipeline():
    rf_classifier = RandomForestClassifier(labelCol = 'canceled', featuresCol = 'features', seed = 42)
    return create_pipeline(rf_classifier)

def create_gradient_boost_pipeline():
    gbt_classifier = GBTClassifier(labelCol = 'canceled', maxDepth = 5, maxIter = 100)
    return create_pipeline(gbt_classifier)

In [16]:
# Create the new dataframe
df_users = df_users.select(columns_all).fillna(0)

In [ ]:
df_users.show(1, False)

In [ ]:
model, predictions = train_random_forest_classifier(df_users, columns_all, columns_to_train)

In [ ]:
evaluate_multiclass_classifier(predictions, ('canceled', 'prediction'))

In [ ]:
df_results = predictions.select(['canceled', 'prediction', 'predictedLabel']).toPandas()
df_results['prediction'] = df_results.prediction.apply(int)
plot_confusion_matrix(df_results['canceled'], df_results['prediction'])

In [ ]:
plot_confusion_matrix(df_results['canceled'], df_results['prediction'])

In [ ]:
plot_confusion_matrix(df_results['canceled'], df_results['prediction'])

In [17]:
from sklearn.metrics import confusion_matrix

In [18]:
import numpy as np

# plot_confusion_matrix(df_results['canceled'], df_results['prediction'])

In [19]:
def plot_confusion_matrix(y_test, y_predictions):
    
    # auc = roc_auc_score(y_test, y_predictions)
    cm = confusion_matrix(y_test, y_predictions, labels = [1, 0])
    
    tn = cm[0, 0]
    tp = cm[1, 1]
    fn = cm[1, 0]
    fp = cm[0, 1]
    
    total = np.sum(cm) # tn + tp + fn + fp
    accuracy = (tp + tn) / total
    precision = (tp) / (tp + fp)
    recall = (tp) / (tp + fn) 
    
    print(cm)

In [ ]:
model, predictions = train_random_forest_classifier(df_users, columns_to_use, columns_to_train)

In [ ]:
evaluate_multiclass_classifier(predictions, ('canc', 'prediction'))

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol = 'canc', metricName = 'areaUnderROC')

evaluator.evaluate(predictions)

In [ ]:
maxBins = 5, impurity = 'entropy', minInstancesPerNode = 3, seed = 42

In [15]:
list(range(5, 45, 5))

[5, 10, 15, 20, 25, 30, 35, 40]

In [20]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

def create_grid_search(pipeline, param_grid, evaluator = MulticlassClassificationEvaluator(labelCol = 'canceled', metricName = 'f1')):
    
    return CrossValidator(estimator = pipeline, estimatorParamMaps = param_grid, evaluator = evaluator, numFolds = 2, parallelism = 12)

def random_forest_grid_search(pipeline):
    
    model = pipeline.getStages()[-1]

    grid_rf = ParamGridBuilder().addGrid(model.maxDepth, [5, 10, 15, 20, 25]) 
    grid_rf = grid_rf.addGrid(model.impurity, ['gini']) 
    grid_rf = grid_rf.addGrid(model.maxBins, [5, 10, 15, 20, 25, 30, 35, 40])
    grid_rf = grid_rf.addGrid(model.numTrees, [10, 20, 40, 60, 70])
    grid_rf = grid_rf.build()
    
    print(f'Number of models to train: {len(grid_rf)}')
        
    return create_grid_search(pipeline, grid_rf)

def gradient_boost_grid_search(pipeline):
    
    model = pipeline.getStages()[-1]

    grid_gbt = ParamGridBuilder().addGrid(model.maxDepth, [5]) #, 10, 15, 20, 25])
    grid_gbt = grid_gbt.addGrid(model.maxIter, [20])#, 25, 40, 50, 100])
    grid_gbt = grid_gbt.build()
   
    return create_grid_search(pipeline, grid_gbt)

In [21]:
# Create the new dataframe
# data = df_users.select(columns_to_use).fillna(0)

# Split train/test
(train_df, test_df) = df_users.randomSplit(SPLIT_RATIO, seed = 42)

In [22]:
pipeline = create_random_forest_pipeline()
cv_rf = random_forest_grid_search(pipeline)
cv_rf_results = cv_rf.fit(train_df)

Number of models to train: 200


In [48]:
dir(cv_rf_results)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__metaclass__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_clear',
 '_copyValues',
 '_copy_params',
 '_defaultParamMap',
 '_dummy',
 '_from_java',
 '_from_java_impl',
 '_paramMap',
 '_params',
 '_randomUID',
 '_resetUid',
 '_resolveParam',
 '_set',
 '_setDefault',
 '_shouldOwn',
 '_to_java',
 '_to_java_impl',
 '_transform',
 'avgMetrics',
 'bestModel',
 'copy',
 'estimator',
 'estimatorParamMaps',
 'evaluator',
 'explainParam',
 'explainParams',
 'extractParamMap',
 'getEstimator',
 'getEstimatorParamMaps',
 'getEvaluator',
 'getOrDefault',
 'getParam',
 'getSeed',
 'hasDefault',
 'hasParam',
 'isDefined',
 'isSet',
 'load',
 'params',
 'read',
 'save',
 'seed

In [47]:
cv_rf_results.avgMetrics

[0.8106176223201276,
 0.8044321902562253,
 0.8025702525429657,
 0.829491736551097,
 0.8219670917806259,
 0.8034024734959226,
 0.8155870376145662,
 0.8180662175976634,
 0.81007777035447,
 0.8332143407541575,
 0.7926130952316295,
 0.8172692828597874,
 0.815413940543019,
 0.815413940543019,
 0.8236515615130073,
 0.8120970997802575,
 0.8131841867250467,
 0.829491736551097,
 0.8240961423694948,
 0.8303445788788958,
 0.8123558512528858,
 0.8285646538451237,
 0.816374624590775,
 0.836648714022421,
 0.8218097356904649,
 0.7977407046367286,
 0.8218252347100499,
 0.8183961601568988,
 0.8233658771672054,
 0.8318330274288643,
 0.7986710075301835,
 0.824874704986463,
 0.829217769367822,
 0.8207574829082216,
 0.8341832533115061,
 0.7962735066707687,
 0.8058926802836588,
 0.7879022766934174,
 0.8287041495748881,
 0.8303494444211224,
 0.8038146273579008,
 0.8193808370173861,
 0.8048641495748883,
 0.8257886948146709,
 0.8229082355963935,
 0.7981768464321646,
 0.815939694095531,
 0.8224027777544556,
 0.

In [46]:
# len(cv_rf_results.getEstimatorParamMaps())

list(zip(cv_rf_results.avgMetrics, cv_rf_results.getEstimatorParamMaps()))


[(0.8106176223201276,
  {Param(parent='RandomForestClassifier_f4ca53297116', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
   Param(parent='RandomForestClassifier_f4ca53297116', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini',
   Param(parent='RandomForestClassifier_f4ca53297116', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 5,
   Param(parent='RandomForestClassifier_f4ca53297116', name='numTrees', doc='Number of trees to train (>= 1).'): 10}),
 (0.8044321902562253,
  {Param(parent='RandomForestClassifier_f4ca53297116', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
   Param(parent='RandomForestClassifier_f4ca532

In [51]:
pipeline = create_random_forest_pipeline()
cv_rf = random_forest_grid_search(pipeline)
cv_rf_results = cv_rf.fit(train_df)

Number of models to train: 200


Py4JJavaError: An error occurred while calling o265827.transform.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 11 in stage 18510.0 failed 1 times, most recent failure: Lost task 11.0 in stage 18510.0 (TID 1531258, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1887)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1875)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1874)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1874)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scal

In [23]:
predictions = cv_rf_results.bestModel.transform(test_df)

df_results = predictions.select(['canceled', 'prediction']).toPandas()
df_results['prediction'] = df_results.prediction.apply(int)

plot_confusion_matrix(df_results['canceled'], df_results['prediction'])

[[16 15]
 [ 5 96]]


In [32]:
cv_rf_results.bestModel.stages[-1].numFeatures

33

In [ ]:
rf = RandomForestClassifier(labelCol = 'idx_labels', featuresCol = 'features', numTrees = 10)
dir(rf)

In [24]:
cv_rf_results.bestModel

PipelineModel_85311fec7e03

In [ ]:
df_results = predictions.select(['canceled', 'prediction']).toPandas()
df_results['prediction'] = df_results.prediction.apply(int)
plot_confusion_matrix(df_results['canceled'], df_results['prediction'])

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol = 'canc', metricName = 'areaUnderROC')
 
best_model_results = cv_gbt_results.bestModel.transform(test_df)
    
evaluator.evaluate(best_model_results)

In [ ]:
pipeline = create_gradient_boost_pipeline()
cv_gbt = gradient_boost_grid_search(pipeline)

In [ ]:
cv_gbt_results = cv_gbt.fit(train_df)

In [ ]:
cv_gbt_results.bestModel

In [ ]:
import pandas as pd

scores = cv_gbt_results.avgMetrics
params = [{p.name: v for p, v in m.items()} for m in cv_gbt.getEstimatorParamMaps()]
params_pd = pd.DataFrame(params)
params_pd['score'] = scores
params_pd

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol = 'canc', metricName = 'areaUnderROC')
 
best_model_results = cv_gbt_results.bestModel.transform(test_df)
    
evaluator.evaluate(best_model_results)

In [ ]:
# evaluator = BinaryClassificationEvaluator(labelCol = 'canceled', metricName = 'f1Measure')

metrics_to_evaluate = [ 'accuracy', 'f1', 'weightedPrecision', 'weightedRecall' ]

evaluator = MulticlassClassificationEvaluator(labelCol = 'canceled', metricName = 'f1')
 
best_model_results = cv_rf_results.bestModel.transform(test_df)
    
evaluator.evaluate(best_model_results)

In [ ]:
evaluate_multiclass_classifier(best_model_results, ('canc', 'prediction'))

In [ ]:
evaluate_multiclass_classifier(best_model_results, ('canc', 'prediction'))

In [ ]:
best_model_results.select(['features', 'prediction', 'canc']).show(10, False)

In [ ]:
best_model_results.select(['rawPrediction', 'prediction', 'canc']).show(10, False)

In [ ]:
test_df.filter('canc = 1').count()

In [ ]:
train_df.filter('canc = 1').count()

In [ ]:
best_model_results.select("prediction", "canc", "features").filter('canc = 1').groupby(['canc', 'prediction']).agg({'canc':'count'}).show(50)

In [ ]:
cv_rf_results.bestModel.stages[-1]

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.